In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import csv
import datetime as dt
import random
import numpy as np

In [ ]:
#Contains play by play data from 2015-2023 NFL seasons
df_list = ["pbp_2015.csv", "pbp_2016.csv", "pbp_2017.csv", "pbp_2018.csv", "pbp_2019.csv", "pbp_2020.csv", "pbp_2021.csv", "pbp_2022.csv", "pbp_2023.csv"]

In [ ]:
#Must get above files from nfl_data_py package in README. Import by the following code

!pip install nfl_data_py
import nfl_data_py as nfl
x=nfl.import_pbp_data(years=[2022], downcast=True, cache=False, alt_path=None)
x.to_csv("test.csv")

In [ ]:
"""
    Creates variables xp_attempt_perc, xp2_attempt_perc, xp_make_perc, xp2_make_perc
    Stored in arrays based on score difference
    
    i.e. xp_attempt_perc[-8] returns the real percentage that teams down 8 attempt an extra 
        point from the 2015-2023 NFL seasons. Similarly, xp_make_perc[-8] shows the percentage
        of successful extra points when a team is down 8 in the 2015-2023 seasons. xp2_attempt_perc
        and xp2_make_perc work the same way storing information on 2 point conversions. 
    
    Parameters:
    - None

    Returns:
    - None
"""

#Initializes each value to 0. Score difference spans from [-70 to 70].
ScoreDiff = [x for x in range(-70, 71)]
XP_Attempt_Count={}
XP2_Attempt_Count={}
for sd in ScoreDiff:
    XP_Attempt_Count[sd] = 0
    XP2_Attempt_Count[sd] = 0

xp_attempt_perc = {}
xp2_attempt_perc = {}

xp_makes=0
xp2_makes=0

for df in df_list:

    df = pd.read_csv(df)

    for index, row in df.iterrows():

        #Counts for each attempt and make for both extra points and two point conversions
        
        if(row['extra_point_attempt']==1):
            XP_Attempt_Count[int(row['score_differential_post'])]+=1
        if(row['two_point_attempt']==1):
            XP2_Attempt_Count[int(row['score_differential_post'])]+=1
        if(row['extra_point_result']=='good'):
            xp_makes+=1
        if(row['two_point_conv_result']=='success'):
            xp2_makes+=1


    #Calculate attempt percentages at each score difference for both extra points and two point conversions
    for sd in XP_Attempt_Count:
        try:
            xp_attempt_perc[sd] = XP_Attempt_Count[sd]/(XP_Attempt_Count[sd]+XP2_Attempt_Count[sd])
            xp2_attempt_perc[sd] = XP2_Attempt_Count[sd]/(XP_Attempt_Count[sd]+XP2_Attempt_Count[sd])

        except ZeroDivisionError:
            XP_Attempt_Count[sd]=0
            XP2_Attempt_Count[sd]=0

#Calcualte overall make percentage
xp_make_perc = xp_makes/sum(XP_Attempt_Count.values())
xp2_make_perc = xp2_makes/sum(XP2_Attempt_Count.values())

In [ ]:
'''
Creates dictionary with keys of all overtime final scores
Note 2023 rules allow for possible other variations, but it has not occurred

    Parameters:
    - None

    Returns:
    - None
'''

ot = pd.read_csv("All NFL Game Data (Including OT Stats).csv")

keylist = ['0-0', '2-0', '3-0', '6-0', '3-3', '6-3']
ot_count=0
ot_scores = {}
for i in keylist:
    ot_scores[i] = 0

#Counts the total of each key
for index, row in ot.iterrows():
    if (int(row['SEASON'])>=2015 and row['OVERTIME']):
        ot_count+=1
        ot_scores[row['OT SCORE']]+=1

#Divides by 2 as the csv has the final score for both teams 
for score in ot_scores:
    ot_scores[score]=ot_scores[score]/2

In [ ]:
"""
Simulates Overtime

Selects the scoring team based off weights from all scoring events. Then selects the final score based off percentages from the
2015-2023 seasons. 

Parameters:
- t1 (int): Rank of the first team
- t2 (int): Rank of the second team
- avg_mu (list): Array of average scoring mus -avg_mu[rank][period][event]

Returns:
- (tuple): Returns the selected scoring team and the final score after overtime
"""

def ot_scoring(t1, t2, avg_mu):

    w1=0
    w2=0
    for period in range(6):
        w1+=(avg_mu[t1][period][0]+avg_mu[t1][period][1]+avg_mu[t1][period][2]+avg_mu[t1][period][3]+avg_mu[t1][period][4]+avg_mu[t1][period][5])
        w2+=(avg_mu[t2][period][0]+avg_mu[t2][period][1]+avg_mu[t2][period][2]+avg_mu[t2][period][3]+avg_mu[t2][period][4]+avg_mu[t2][period][5])

    weight1=w1/(w1+w2)
    weight2=1-weight1

    scoring_team = random.choices([1,2], weights=(weight1, weight2))
    scoring_team = scoring_team[0]


    ot_event = random.choices([1,2,3,4,5,6], weights=(ot_scores['0-0']/ot_count, ot_scores['2-0']/ot_count, ot_scores['3-0']/ot_count, ot_scores['6-0']/ot_count, ot_scores['3-3']/ot_count, ot_scores['6-3']/ot_count))
    ot_event = ot_event[0]


    if ot_event == 1:
        return (scoring_team, [0,0])
    if ot_event == 2:
        return (scoring_team, [2,0])
    if ot_event == 3:
        return (scoring_team, [3,0])
    if ot_event == 4:
        return (scoring_team, [6,0])
    if ot_event == 5:
        return (scoring_team, [3,3])
    if ot_event == 6:
        return (scoring_team, [6,3])

In [ ]:
"""
Stores information on team scoring events and period for every season

i.e.'LAC': [11, 11, 11, 9, 15, 4] means the Chargers had 11 scoring events in the first qtr, 11 scoring events in the second qtr - 2 minute warning, etc.

    Parameters:
    - None

    Returns:
    - None
"""

team_list = ['LAC', 'TEN', 'JAX', 'TB', 'HOU', 'LA', 'LV', 'NE', 'BUF', 'CIN', 'BAL', 'PIT', 'ARI', 'ATL', 'NO', 'IND', 'KC', 'CAR', 'MIA', 'PHI', 'DAL', 'SF', 'SEA', 'WAS', 'DEN', 'NYJ', 'CLE', 'MIN', 'DET', 'CHI', 'NYG', 'GB']
df_list = ["pbp_2015.csv", "pbp_2016.csv", "pbp_2017.csv", "pbp_2018.csv", "pbp_2019.csv", "pbp_2020.csv", "pbp_2021.csv", "pbp_2022.csv", "pbp_2023.csv"]

period_dict_list=[]

year_count=2015

for df in df_list:

    df = pd.read_csv(df)

    team_period_dict={}

    for team in team_list:
        team_period_dict[team]=[0,0,0,0,0,0]

    for index, row in df.iterrows():

        team=''

        if row['season_type']=='REG':

            if ((row['touchdown'] == 1 or row['field_goal_result']=='made' or row['safety']==1 or row['defensive_two_point_conv']==1)):

                if row['posteam_score_post']>row['posteam_score']:
                    team='posteam'
                elif row['defteam_score_post']>row['defteam_score']:
                    team='defteam'


                if(row['qtr']==1):
                    team_period_dict[row[team]][0]+=1

                if(row['qtr']==2):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][1]+=1
                    else:
                        team_period_dict[row[team]][2]+=1

                if(row['qtr']==3):
                    team_period_dict[row[team]][3]+=1


                if(row['qtr']==4):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][4]+=1
                    else:
                        team_period_dict[row[team]][5]+=1


    period_dict_list.append(team_period_dict)
    year_count+=1

In [ ]:
"""
Stores information on total scoring events per team in a season

i.e. {2015: [(('LAC'), 61)] means the Chargers had 61 total scoring events in 2015

    Parameters:
    - None

    Returns:
    - None
"""

team_list = period_dict_list[0].keys()

team_total_scoring_events = {
    2015: [],
    2016: [],
    2017: [],
    2018: [],
    2019: [],
    2020: [],
    2021: [],
    2022: [],
    2023: []
}

season_count=0
for key in team_total_scoring_events:

    for team in team_list:

        team_total_scoring_events[key]+=[(team, sum(period_dict_list[season_count][team]))]

    season_count+=1


In [ ]:
"""
Stores information on scoring events per team per period in a season

i.e. 'LAC': [[1, 4, 0, 6, 0, 0], [1, 5, 0, 5, 0, 0], [1, 5, 0, 5, 0, 0], [0, 6, 0, 3, 0, 0], [2, 7, 1, 5, 0, 0], [0, 1, 1, 2, 0, 0]]
means that the Chargers had 1 TD6, 4 TD7, 0 TD8, 6 FG, 0 SFTY, and 0 D2P in the first quarter, etc.

    Parameters:
    - None

    Returns:
    - None
"""

team_list = ['LAC', 'TEN', 'JAX', 'TB', 'HOU', 'LA', 'LV', 'NE', 'BUF', 'CIN', 'BAL', 'PIT', 'ARI', 'ATL', 'NO', 'IND', 'KC', 'CAR', 'MIA', 'PHI', 'DAL', 'SF', 'SEA', 'WAS', 'DEN', 'NYJ', 'CLE', 'MIN', 'DET', 'CHI', 'NYG', 'GB']
df_list = ["pbp_2015.csv", "pbp_2016.csv", "pbp_2017.csv", "pbp_2018.csv", "pbp_2019.csv", "pbp_2020.csv", "pbp_2021.csv", "pbp_2022.csv", "pbp_2023.csv"]

period_dict_list=[]

year_count=2015
event_index=0
scoring_event=False
team=''

for df in df_list:

    df = pd.read_csv(df)

    team_period_dict={}

    for team in team_list:
        team_period_dict[team]=[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

    for index, row in df.iterrows():

        scoring_event=False

        if row['season_type']=='REG':

            #TD 6
            if ((row['extra_point_result'] == 'failed' or row['extra_point_result'] == 'blocked' or row['two_point_conv_result']=='failure')):
                event_index=0
                scoring_event=True

            #TD 7
            if (row['extra_point_result'] == 'good'):
                event_index=1
                scoring_event=True

            #TD8
            if(row['two_point_conv_result']=='success'):
                event_index=2
                scoring_event=True

            #FG
            if (row['field_goal_result']=='made'):
                event_index=3
                scoring_event=True

            #SFTY
            if (row['safety']==1):
                event_index=4
                scoring_event=True

            #DP2
            if (row['defensive_two_point_conv']==1):
                event_index=5
                scoring_event=True

            if scoring_event==True:


                if row['defteam_score_post']>row['defteam_score']:
                    team='defteam'
                else:
                    team='posteam'

                if(row['qtr']==1):
                    team_period_dict[row[team]][0][event_index]+=1

                if(row['qtr']==2):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][1][event_index]+=1
                    else:
                        team_period_dict[row[team]][2][event_index]+=1

                if(row['qtr']==3):
                    team_period_dict[row[team]][3][event_index]+=1

                if(row['qtr']==4):
                    time=(row['time'].split(':'))
                    if (int(time[0])>=2):
                        team_period_dict[row[team]][4][event_index]+=1
                    else:
                        team_period_dict[row[team]][5][event_index]+=1


    period_dict_list.append(team_period_dict)
    year_count+=1

In [ ]:
'''
The overall scoring event rate in year y in period p of scoring event type e for the r-th best team

2015 mu[year(2015-2023)][ranking(1-32)][period(1-6][event(1-6)]

mu[2015][0][0][0] show best TD6 scoring rate in first quarter in 2015

    Parameters:
    - None

    Returns:
    - None
'''

mu = [[[[0 for _ in range(6)] for _ in range(6)] for _ in range(32)] for _ in range(9)]

for year in range(9):
        for period in range(6):
            for event in range(6):
                sorted_list = sorted(period_dict_list[year].items(), key=lambda x: x[1][period][event], reverse=True)
                for rank in range(32):
                    mu[year][rank][period][event]=sorted_list[rank][1][period][event]

In [ ]:
'''
The overall scoring event rate in period p of scoring event type e for the r-th best team

2015 mu[ranking(1-32)][period(1-6][event(1-6)]

mu[0][0][0] show best TD6 scoring rate in first quarter in 2015

    Parameters:
    - None

    Returns:
    - None
'''

avg_mu = [[[0 for _ in range(6)] for _ in range(6)] for _ in range(32)]
games=0


for period in range(6):
    for event in range(6):
        for rank in range(32):
            for year in range(9):

                avg_mu[rank][period][event] += mu[year][rank][period][event]

            avg_mu[rank][period][event] = avg_mu[rank][period][event]/147

In [ ]:
'''
Get Average Scoring Rate Per Period
    Parameters:
    - None

    Returns:
    - None
'''

period1_total = 0
period2_total = 0
period3_total = 0
period4_total = 0
period5_total = 0
period6_total = 0

for rank in range(32):
    period1_total+=avg_mu[rank][0][0]+avg_mu[rank][0][1]+avg_mu[rank][0][2]+avg_mu[rank][0][3]+avg_mu[rank][0][4]+avg_mu[rank][0][5]
    period2_total+=avg_mu[rank][1][0]+avg_mu[rank][1][1]+avg_mu[rank][1][2]+avg_mu[rank][1][3]+avg_mu[rank][1][4]+avg_mu[rank][1][5]
    period3_total+=avg_mu[rank][2][0]+avg_mu[rank][2][1]+avg_mu[rank][2][2]+avg_mu[rank][2][3]+avg_mu[rank][2][4]+avg_mu[rank][2][5]
    period4_total+=avg_mu[rank][3][0]+avg_mu[rank][3][1]+avg_mu[rank][3][2]+avg_mu[rank][3][3]+avg_mu[rank][3][4]+avg_mu[rank][3][5]
    period5_total+=avg_mu[rank][4][0]+avg_mu[rank][4][1]+avg_mu[rank][4][2]+avg_mu[rank][4][3]+avg_mu[rank][4][4]+avg_mu[rank][4][5]
    period6_total+=avg_mu[rank][5][0]+avg_mu[rank][5][1]+avg_mu[rank][5][2]+avg_mu[rank][5][3]+avg_mu[rank][5][4]+avg_mu[rank][5][5]

period1_mu = 2 * 1/32*(period1_total)
period2_mu = 2 * 1/32*(period2_total)
period3_mu = 2 * 1/32*(period3_total)
period4_mu = 2 * 1/32*(period4_total)
period5_mu = 2 * 1/32*(period5_total)
period6_mu = 2 * 1/32*(period6_total)

In [ ]:
"""
Simulation

Simulates an NFL game by generating a specific number of scoring events in each period.
The period mus are acquired from percentages of real scoring times from 2015-2023.

Parameters:
- avg_mu (list): Array of average scoring mus -avg_mu[rank][period][event]

Returns:
-[s1,s2] (list): Returns a list of one final score in format [team1_score, team2_score]
"""

def sim(avg_mu):


        stoplist = [15, 28, 30, 45, 58, 60]
        numbers = list(range(0, 32))
        random.shuffle(numbers)

        t1=numbers[0]
        t2=numbers[1]

        t=0
        s1=0
        s2=0
        x=0

        for time in stoplist:

            scoring_events=0

            #First Quarter
            if (time==15):
                x = np.random.poisson(period1_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=0)
                    scoring_events+=1

            #Second Quarter
            elif (time==28):
                x = np.random.poisson(period2_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=1)
                    scoring_events+=1


            #Second Quarter 2 min remaining
            elif (time==30):
                x = np.random.poisson(period3_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=2)
                    scoring_events+=1

            #Third Quarter
            elif (time==45):
                x = np.random.poisson(period4_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=3)
                    scoring_events+=1


            #Fourth Quarter
            elif (time==58):
                x = np.random.poisson(period5_mu)

                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=4)
                    scoring_events+=1


            #Fourth Quarter 2 minutes remaining
            elif (time==60):

                x = np.random.poisson(period6_mu)
                while scoring_events < x:
                    [s1, s2] = score_event_sd(s1, s2, t1, t2, avg_mu, period=5)
                    scoring_events+=1

        if (time==60 and s1 != s2):
            return([s1,s2])

        else:
            ot_score_updated = ot_scoring(t1, t2, avg_mu)

            #T1 Scores
            if ot_score_updated[0]==1:
                s1 += ot_score_updated[1][0]
                s2 += ot_score_updated[1][1]
            else:
                s1 += ot_score_updated[1][1]
                s2 += ot_score_updated[1][0]
            return([s1,s2])

In [ ]:
"""
Score Event Simulation

Generates which team should score and how much their points increase by

Parameters:
- s1 (int): Score of team 1
- s2 (int): Score of team 2
- t1 (int): Rank of team 1
- t2 (int): Rank of team 2
- avg_mu (list): Array of average scoring mus -avg_mu[rank][period][event]
- period (int): Current game period (1st quarter, 2nd quarter, 2nd quarter 2 min remaining, etc.)

Returns:
-[s1,s2] (list): Returns a list of updated score in format [team1_score, team2_score]
"""

def score_event_sd(s1, s2, t1, t2, avg_mu, period):

    td_rate=0
    fg_rate=0
    sfty_rate=0

    #Time Remaining at start of each period
    time_remaining_list = [60, 45, 32, 30, 13, 2]
    time_remaining = time_remaining_list[period]

    #Hard coded values of delta and lambda
    delta=0.4
    lamb=0.02

    theta = pow(2.718281828459045, ((lamb*-1)*(60-time_remaining)))

    # Calculating Total TD, FG, and SFTY rates over all teams for current period
    for x in range(32):
        td_rate+=avg_mu[x][period][0]+avg_mu[x][period][1]+avg_mu[x][period][2]+avg_mu[x][period][5]
        fg_rate+=avg_mu[x][period][3]
        sfty_rate+=avg_mu[x][period][4]


    #Get avg scoring rate
    td_rate=td_rate/32
    fg_rate=fg_rate/32
    sfty_rate=sfty_rate/32

    #Create mu_bar (average scoring rate of all events in current period)
    mu_bar = 0
    for team in range(32):
        for event in range(6):
            mu_bar+=(avg_mu[team][period][event])
    mu_bar = (mu_bar*1/32)

    #Mu 1 (average scoring rate for all events for team 1 in current period)
    mu_1=0
    for event in range(6):
        mu_1+=avg_mu[t1][period][event]

    #Mu 2 (average scoring rate for all events for team 2 in current period)
    mu_2=0
    for event in range(6):
        mu_2+=avg_mu[t2][period][event]

    w1=0
    w2=0

    #If t1 is beating t2, give advantage to t2
    if s1>s2:
        w1 = mu_1*theta+(1-theta)*mu_bar*(1-delta)
        w2 = mu_2*theta+(1-theta)*mu_bar*(1+delta)

    #If t2 is beating t1, give advantage to t1
    elif s1<s2:
        w1 = mu_1*theta+(1-theta)*mu_bar*(1+delta)
        w2 = mu_2*theta+(1-theta)*mu_bar*(1-delta)

    #If tied, no advantage
    else:
        w1 = mu_1*theta+(1-theta)*mu_bar
        w2 = mu_2*theta+(1-theta)*mu_bar

    weight1=(w1/(w1+w2))
    weight2=1-weight1

    #"Flip Coin" to decide which team scores
    scoring_team = random.choices([1,2], weights=(weight1, weight2))
    scoring_team = scoring_team[0]

    XP_or_2PT=0

    #Choose scoring event based off above rates
    event = random.choices(["FG", "TD", "SFTY"], weights=(fg_rate, td_rate, sfty_rate))
    event=event[0]

   #Field Goal
    if(event=='FG'):
        if (scoring_team==1):
            return[s1+3, s2]
        else:
            return[s1, s2+3]

    #Safety
    elif(event=='SFTY'):
        if (scoring_team==1):
            return[s1+2, s2]
        else:
            return[s1, s2+2]

    #Touchdown
    else:

        if(scoring_team==1):

            #If score difference does not exist kick xp
            if((s1+6)-s2 not in xp_attempt_perc.keys()):
                XP_or_2PT=1

            else:
                #Deciding 1 or 2 point
                XP_or_2PT_Choice = random.choices([1,2], weights=(xp_attempt_perc[(s1+6)-s2], xp2_attempt_perc[(s1+6)-s2]))
                XP_or_2PT=XP_or_2PT_Choice[0]

            #If 1 point, calculate make rate
            if (XP_or_2PT == 1):
                make_xp = random.choices([1,0], weights=(xp_make_perc, 1-xp_make_perc))
                make_xp=make_xp[0]
                return [s1+6+make_xp, s2]

           #If 2 points, calculate make rate
            elif (XP_or_2PT == 2):
                make_xp2 = random.choices([2,0], weights=(xp2_make_perc, 1-xp2_make_perc))
                make_xp2=make_xp2[0]
                return [s1+6+make_xp2, s2]

        #Team 2 Scores
        else:

            #If score difference does not exist kick xp
            if((s2+6)-s1 not in xp_attempt_perc.keys()):
                XP_or_2PT=1

            else:
                #Deciding 1 or 2 point
                XP_or_2PT_Choice = random.choices([1,2], weights=(xp_attempt_perc[(s2+6)-s1], xp2_attempt_perc[(s2+6)-s1]))
                XP_or_2PT=XP_or_2PT_Choice[0]

            #If 1 point, calculate make rate
            if (XP_or_2PT == 1):
                make_xp = random.choices([1,0], weights=(xp_make_perc, 1-xp_make_perc))
                make_xp=make_xp[0]
                return [s1, s2+6+make_xp]


            #If 2 points, calculate make rate
            elif (XP_or_2PT == 2):
                make_xp2 = random.choices([2,0], weights=(xp2_make_perc, 1-xp2_make_perc))
                make_xp2=make_xp2[0]
                return [s1, s2+6+make_xp2]

    return [s1, s2]

In [ ]:
"""
Runs Many Simulations

Change numsims to produce desire number of simulations
Writes to CSV to analyze data

"""

numsims=300
score_count = [ [0 for i in range(j)] for j in range(1, 200)]

for x in range(300):

    game=sim(avg_mu)

    if(x%50==0):
        print(game)

    win_pts=max(game)
    lose_pts=min(game)

    score_count[win_pts][lose_pts]+=1

with open('ScoreDiff_Sim.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    for i in range(len(score_count)):
        writer.writerow(score_count[i])